In [22]:
var _    = require('lodash')
var fs   = require('fs')
var turf = require('../node_modules/@turf/turf')
var pointsInPolygon = require('../node_modules/@turf/points-within-polygon')

Load user metadata and evacuation zone information to determine if a user's home location is in an evacuation zone

In [23]:
// Harvey
//var EVACUATION_ZONE = './EvacuationZones/HarveyIrmaMaria/Texas-EvacuationCounties.geojson'
//var USER_META_DIR   = '/data/chime/geo2/NEW/HARVEY2'

// //Irma
// var EVACUATION_ZONE = './EvacuationZones/HarveyIrmaMaria/Miami-DadeABC.geojson'
// var USER_META_DIR   = '/data/chime/geo2/NEW/IRMA-STORMPATH2'

//Sandy Zone A
var EVACUATION_ZONE = './EvacuationZones/Sandy/ZoneA.geojson'
var USER_META_DIR   = '/data/chime/geo2/NEW/NYC_ZONEA'

In [24]:
EVACUATION_AREA = JSON.parse(fs.readFileSync(EVACUATION_ZONE))
console.warn(EVACUATION_AREA.features.length)
var EVACUATION_AREA = EVACUATION_AREA.features[0]

1


Load all the user metadata files

In [25]:
var userMetas = []
fs.readdirSync(USER_META_DIR).forEach(function(fileName){
  if (fileName.endsWith('-meta.geojson')){
    var user = JSON.parse(fs.readFileSync(USER_META_DIR+"/"+fileName))
    if (user.features.length==1){
        return false;
    }
    user.features.forEach(function(cluster){
      if(cluster.properties.hasOwnProperty('likelyHome')){
        userMetas.push({type:"Feature",
                        properties: {user : fileName.substring(0,fileName.length-13)},
                        geometry: cluster.geometry});
      }
    })
  }
})
console.warn("Loaded " + userMetas.length + " users with likely Home clusters")

Loaded 135 users with likely Home clusters


In [26]:
var geographicallyVulnerableUsers = pointsInPolygon({type: "FeatureCollection", features: userMetas}, EVACUATION_AREA)
console.warn("Found " + geographicallyVulnerableUsers.features.length + " geographically vulnerable users")

Found 4 geographically vulnerable users


In [27]:
geographicallyVulnerableUsers.features.forEach(function(u){
    console.log(u.properties.user)
})

KevyKevsaunders
LaLaHearts
WizbyF
tlal2


In [21]:
fs.writeFileSync('/data/chime/geo2/NEW/SANDY-ZoneA-geovulnerable.geojson',JSON.stringify(geographicallyVulnerableUsers))

In [29]:
fs.writeFileSync('/data/chime/geo2/SANDY-ZoneA-homes.geojson',JSON.stringify({type: "FeatureCollection", features: userMetas}))